In [1]:
import openmc
import openmc.mgxs
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas
import h5py
%matplotlib inline

In [2]:
import os

In [3]:
os.environ['PATH'] += r':/home/jae123p/miniconda3/envs/openmc-env/bin'

%env OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml


In [4]:
model = openmc.Model()

In [5]:
oralloy = openmc.Material(name="Oralloy")
oralloy.add_nuclide('U234', 4.9210e-04)
oralloy.add_nuclide('U235', 4.4917e-02)
oralloy.add_nuclide('U238', 2.5993e-03)

tuballoy = openmc.Material(name="Tuballoy")
tuballoy.add_nuclide('U234', 2.6299e-06)
tuballoy.add_nuclide('U235', 3.4428e-04)
tuballoy.add_nuclide('U238', 4.7470e-02)

materials = openmc.Materials([oralloy, tuballoy])
materials.export_to_xml('HEU_MET_FAST-003-CASE-1.xml')
model.materials = openmc.Materials([oralloy, tuballoy])

In [6]:
#Geometry
inner_radius = 6.7820
outer_radius = 11.8620

oralloy_sphere = openmc.Sphere(0, 0, 0, r=inner_radius)
tuballoy_sphere = openmc.Sphere(0, 0, 0, r=outer_radius, boundary_type = 'vacuum')


In [7]:
#Regions
or_region = -oralloy_sphere
tub_region = +oralloy_sphere & - tuballoy_sphere

#Cells
or_cell = openmc.Cell(name='or', fill = oralloy, region = or_region)
tub_cell  = openmc.Cell(name='tub', fill = tuballoy, region = tub_region)

geometry = openmc.Geometry([or_cell, tub_cell])
geometry.export_to_xml()

In [8]:
root_universe = openmc.Universe(cells = (or_cell, tub_cell))
model.geometry = openmc.Geometry(root_universe)
cell = openmc.Cell(name = 'cell',fill = root_universe)

In [9]:
#Settings 
settings = openmc.Settings()
settings.batches = 3000
settings.inactive = 20
settings.particles = 10000
settings.runmode = 'eigenvalue'

#Define Source
lower_left = (-1, -1, -1)
upper_right = (1, 1, 1)
uniform_dist = openmc.stats.Box(lower_left, upper_right)
#energy_dist = openmc.stats.Uniform(a=0.0e6,b=20.0e6)
settings.source = openmc.IndependentSource(space=uniform_dist)
settings.export_to_xml()
model.settings = settings


settings.output = {'tallies':False}

In [10]:
my_path = './mgxs_' + cell.name

In [11]:
#Define Energy Groups
groups = openmc.mgxs.EnergyGroups(group_edges='CASMO-40')

In [12]:
# Initialize MGXS Library
mgxs_lib = openmc.mgxs.Library(model.geometry)

# Add the chosen group structure
mgxs_lib.energy_groups = groups

# Scatttering Format and Legendre Order
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 3

# Specify multi-group cross-section types to compute
mgxs_lib.mgxs_types = ['total']
# set uncertainty goal
mgxs_lib.tally_trigger = openmc.Trigger('std_dev', 1e-4)

# Compute cross sections on a nuclide-by-nuclide basis
mgxs_lib.by_nuclide = False

# Specify a "cell" domain type for the cross section tally filters
mgxs_lib.domain_type = 'universe'

# Specify the cell domains over which to compute multi-group cross sections
#mgxs_lib.domains = model.geometry.get_all_cells().values()

# Construct all tallies needed for the multi-group cross section library
mgxs_lib.build_library()

tallies = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies, merge=True)
model.tallies = tallies

/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


In [13]:
#Tallies to Calculate Neutron Flux
energies = openmc.mgxs.GROUP_STRUCTURES['XMAS-172']
XMAS_filter = openmc.EnergyFilter(openmc.mgxs.GROUP_STRUCTURES['XMAS-172'])

tally_XMAS = openmc.Tally()
tally_XMAS.filters = [XMAS_filter]
tally_XMAS.scores = ['flux']

model.tallies.append(tally_XMAS)
model.export_to_model_xml(path = my_path + '/' + cell.name + '.xml')

In [14]:
sp_file = model.run()

[LAPTOP-H8MU6HJU:00695] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.LAPTOP-H8MU6HJU.1000/jf.0/4234346496/shared_mem_cuda_pool.LAPTOP-H8MU6HJU could be created.
[LAPTOP-H8MU6HJU:00695] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%

In [15]:
# Load the last statepoint file
sp = openmc.StatePoint(sp_file)

In [16]:
# Initialize MGXS Library with OpenMC statepoint data
mgxs_lib.load_from_statepoint(sp)

In [17]:
# Store the cross section data in an "mgxs/mgxs.h5" HDF5 binary file
mgxs_lib.build_hdf5_store(filename='case_001.h5')

In [18]:
print(mgxs_lib.domains)

[Universe
	ID             =	2
	Name           =	
	Geom           =	CSG
	Cells          =	[1, 2]
]


In [26]:
# Open the HDF5 file
with h5py.File("mgxs/case_001.h5", "r") as f:
    print("=== File Structure ===")
    
    # Recursively print all groups/datasets
    def print_attrs(name, obj):
        print(f"{name}:")
        if isinstance(obj, h5py.Dataset):
            print(f"  Shape: {obj.shape}, Dtype: {obj.dtype}")
            if obj.shape[0] < 10:  # Print small datasets
                print(f"  Data: {obj[:]}")
        print("  Attributes:")
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")
    
    f.visititems(print_attrs)

=== File Structure ===
universe:
  Attributes:
universe/2:
  Attributes:
universe/2/total:
  Attributes:
universe/2/total/average:
  Shape: (40,), Dtype: float64
  Attributes:
universe/2/total/std. dev.:
  Shape: (40,), Dtype: float64
  Attributes:


In [30]:
# Open the HDF5 file
with h5py.File("mgxs/Oralloy_LANL30g.h5", "r") as f:
    print("=== File Structure ===")
    
    # Recursively print all groups/datasets
    def print_attrs(name, obj):
        print(f"{name}:")
        if isinstance(obj, h5py.Dataset):
            print(f"  Shape: {obj.shape}, Dtype: {obj.dtype}")
            if obj.shape[0] < 10:  # Print small datasets
                print(f"  Data: {obj[:]}")
        print("  Attributes:")
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")
    
    f.visititems(print_attrs)

=== File Structure ===
Oralloy:
  Attributes:
    fissionable: True
    order: 7
    representation: b'isotropic'
    scatter_format: b'legendre'
    scatter_shape: b"[G][G'][Order]"
Oralloy/294K:
  Attributes:
Oralloy/294K/absorption:
  Shape: (30,), Dtype: float64
  Attributes:
Oralloy/294K/chi:
  Shape: (30,), Dtype: float64
  Attributes:
Oralloy/294K/fission:
  Shape: (30,), Dtype: float64
  Attributes:
Oralloy/294K/nu-fission:
  Shape: (30,), Dtype: float64
  Attributes:
Oralloy/294K/scatter_data:
  Attributes:
Oralloy/294K/scatter_data/g_max:
  Shape: (30,), Dtype: int64
  Attributes:
Oralloy/294K/scatter_data/g_min:
  Shape: (30,), Dtype: int64
  Attributes:
Oralloy/294K/scatter_data/multiplicity_matrix:
  Shape: (278,), Dtype: float64
  Attributes:
Oralloy/294K/scatter_data/scatter_matrix:
  Shape: (2224,), Dtype: float64
  Attributes:
Oralloy/294K/total:
  Shape: (30,), Dtype: float64
  Attributes:
Oralloy/kTs:
  Attributes:
Oralloy/kTs/294K:
  Shape: (), Dtype: float64


IndexError: tuple index out of range